In [19]:
from contextlib import redirect_stdout # For printing our files
import os # OS File system utilization in printing files
import pandas as pd #Manipulating our data frames
import numpy as np #Utilizing numpy commands

In [20]:
# IN CASE We want to utilize later cycles of the batteries, we can only use sixgroup. However, all other files have at least one cycle to use, so we can utilize all of the files for the first cycle analysis.
# 1HDC = 1 Hour Dis-Charge
# 3HEC = 3 Hour Equalization Charge
#1HDC, 3HEC, 1HDC
threegroup = [19, 20, 83, 84]

#1HDC, 3HEC, 1HDC, 3HEC
fourgroup = [6, 4, 3, 2, 1]

#1HDC, 3HEC, 1HDC, 3HEC, 1HDC
fivegroup = [95, 96, 94, 93, 65, 54, 53, 31, 30, 29, 15]

#1HDC, 3HEC, 1HDC, 3HEC, 1HDC, 3HEC
sixgroup = [91, 89, 88, 87, 86, 85, 80, 79, 78, 77, 74, 73, 72, 71, 66, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 52, 51, 50, 49, 46, 45, 42, 41, 40, 39, 38, 37, 36, 35, 34, 32, 28, 26, 25, 21, 22, 21, 18, 17, 16,  14, 13]


In [21]:
trurange = list(range(1,69))+list(range(83,99)) # After iterating through which files exist, we have created a range of all possible useable files.
note = {} # Create a dictionary to store our formatted and read files
dischargestart = 'Finish=MaxChargeC' # Start of the discharge cycle, graphed as a plateau in the data
chargestart = 'Finish=CutoffV' # End of the discharge cycle, start of the charge cycle, graphed as a trough in the data
split = 'Finish=MaxDischargeTime' # Split halfway through the charge cycle as current increases, appears as the point where the charge cycle changes.
end = 'Finish=UserRequest:Stop' # End of the data file
for x in trurange:
    try:
        note[x] =(
             pd.read_csv(
                f"NiCd - Key Lime #{x}(data).csv", 
                skip_blank_lines=False, #We want all index numbers to stay the same
                keep_default_na=True, #We want to keep all of our N/A values for the same reason
                names=['Time', 'Volt', 'M/A', 'C', 'Sig'],
            )
        )
    except:
        print(f'fail in {x}')

In [22]:
#We will print out the index locations of all of the signifiers of the first true full discharge & charge cycle. The first cycle in all files is a pre-test that is not used for this experiment.
for x in trurange:
    try:
        if note[x].loc[note[x]['Sig'] == dischargestart].values[0,0] < note[x].loc[note[x]['Sig'] == chargestart].values[1,0] < note[x].loc[note[x]['Sig'] == split].values[2,0]: #Only print files that have their formatting of signatures in succesive order
            print(f'UTILIZING FILE {x}') #To clearly delineate our files being used for each print out 
            print(note[x].loc[note[x]['Sig'] == dischargestart].values[0,0], 'discharge')
            print(note[x].loc[note[x]['Sig'] == chargestart].values[1,0], 'charge')
            print(note[x].loc[note[x]['Sig'] == split].values[2,0], 'split')
    except:
        print('fail')

UTILIZING FILE 2
20369 discharge
21624 charge
22223 split
UTILIZING FILE 3
11856 discharge
12740 charge
13340 split
UTILIZING FILE 4
12344 discharge
13685 charge
14285 split
UTILIZING FILE 5
20530 discharge
21474 charge
22074 split
UTILIZING FILE 6
21349 discharge
22367 charge
22967 split
UTILIZING FILE 7
20234 discharge
21226 charge
21826 split
UTILIZING FILE 8
20567 discharge
21485 charge
22085 split
UTILIZING FILE 9
21732 discharge
22658 charge
23259 split
UTILIZING FILE 10
21533 discharge
22499 charge
23099 split
UTILIZING FILE 11
11984 discharge
13027 charge
13627 split
UTILIZING FILE 12
11667 discharge
12438 charge
13038 split
UTILIZING FILE 13
21371 discharge
22327 charge
22927 split
UTILIZING FILE 14
19291 discharge
20247 charge
20846 split
UTILIZING FILE 15
20046 discharge
20981 charge
21581 split
UTILIZING FILE 16
11982 discharge
13075 charge
13675 split
UTILIZING FILE 17
11968 discharge
13003 charge
13603 split
UTILIZING FILE 18
11757 discharge
12639 charge
13239 split
UTILI

In [23]:
locsigdc = {} #first true discharge positional location
locsigch = {} #first true charge positional location
locsigsp = {} #first true split positional location
locsigdctwo = {} #second true discharge positional location
locsigchtwo = {} #second true charge positional location
locsigsptwo = {} #second true split positional location

for x in note.keys(): 
    try:
        if note[x].loc[note[x]['Sig'] == dischargestart].values[0,0] < note[x].loc[note[x]['Sig'] == chargestart].values[1,0] < note[x].loc[note[x]['Sig'] == split].values[2,0]:
                locsigdc[x] = (note[x].loc[note[x]['Sig'] == dischargestart].values[0,0]) 
                locsigch[x] = (note[x].loc[note[x]['Sig'] == chargestart].values[1,0])
                locsigsp[x] = (note[x].loc[note[x]['Sig'] == split].values[2,0])
    except:
        print(f'{x} fail')


34 fail
67 fail
68 fail
88 fail


In [24]:
dischargecycle = {} #first discharge
splitchargecycle = {} #first half of the charge cycle
postsplit = {} # last half of the charge cycle
for x in range(0,100):
    if x in locsigdc.keys():
        dischargecycle[x] = note[x]['Volt'].truncate(before=locsigdc[x], after=locsigch[x]).tolist()
        splitchargecycle[x] = note[x]['Volt'].truncate(before=locsigch[x], after=locsigsp[x]).tolist()
        if note[x]['Volt'].index[-1] < locsigsp[x]:
            postsplit[x] = (note[x][(note[x]['Sig'] == split).idxmax():])


In [26]:
for x in range(0,100):
    if x in locsigdc.keys():
        try:
            print(postsplit[x]['Volt'].tolist())
        except:
            print(f'Broken Data in #{x}')

[1.34691, 1.14779, 1.13801, 1.13088, 1.12471, 1.11913, 1.11493, 1.11047, 1.10709, 1.10308, 1.09979, 1.09657, 1.09358, 1.09029, 1.08743, 1.08435, 1.0818, 1.07941, 1.07661, 1.07417, 1.07147, 1.06928, 1.06667, 1.06443, 1.0621, 1.05982, 1.05739, 1.05525, 1.05334, 1.05101, 1.04848, 1.04689, 1.04436, 1.0423, 1.0403, 1.03828, 1.03622, 1.0342, 1.03281, 1.03032, 1.02847, 1.02634, 1.02448, 1.02261, 1.02052, 1.01843, 1.01641, 1.01438, 1.01259, 1.01053, 1.00871, 1.00643, 1.00445, 1.00265, 1.00028, 0.998404, 0.995923, 0.993784, 0.991818, 0.989842, 0.987762, 0.98571, 0.983287, 0.981185, 0.978326, 0.975635, 0.972831, 0.96971, 0.966014, 0.962113, 0.958224, 0.953984, 0.94903, 0.943965, 0.93843, 0.932467, 0.925848, 0.918301, 0.910093, 0.898828, 1.14161, 1.16881, 1.18055, 1.18821, 1.19361, 1.19774, 1.20122, 1.20383, 1.20624, 1.20833, 1.21023, 1.21193, 1.21308, 1.21462, 1.2158, 1.21699, 1.21789, 1.21889, 1.21972, 1.22088, 1.42521, 1.44994, 1.46327, 1.47192, 1.47926, 1.48546, 1.49074, 1.49513, 1.4989, 1.50

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [27]:
print(postsplit[27]['Volt'].tolist()) #We have issues with this particular file

KeyError: 27

In [ ]:
#Code made by Paul Williamson/KB5MYU with very small edits by me - much thanks!
user = '' #put your user here if your repo matches with mine
uba_folder = f'/Users/{user}/Downloads/documents-master/Remote_Labs/Project_NiCd/Data/Key Lime' #format for your github repo where needed
uba_extension = ".uba"
output_extension = "_summmary.txt"

for filename in os.listdir(uba_folder):
    if filename.endswith(uba_extension):
        with open(os.path.join(uba_folder, filename), 'r', encoding='cp1252') as f:
            with open(filename+output_extension, 'w') as outf:
                print(filename)
                at_messages = False
                for line in f:
                    if line.strip() == "<Messages>":
                        at_messages = True
                    if at_messages:
                        print(line.strip(), file=outf)